In [2]:
# You may need to run this the first time !pip install lorem confluent_kafka sseclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5570 sha256=4449f89f8f92aa60ce7820ef8e9f2f3c0da52ff00cf8f57176bc8130780cf361
  Stored in directory: /root/.cache/pip/wheels/73/21/3f/9d20c05c1c9312ab61bc7bf63bb2c189a71e800cb028fa8d2a
Successfully built sseclient


In [3]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, lorem, random

brokers = "kafka1:9092,kafka2:9093"

In [4]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer': StringSerializer('utf_8')
}

In [5]:
p = SerializingProducer(pconf)

In [ ]:
import time
for n in range(1,1000):
    try:
        # Produce line (without newline)
        line = lorem.sentence()
        p.produce("words", key=str(uuid4()), value=line)
        p.poll(0)
        p.flush()
        print(line)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

In [ ]:
import time
for n in range(1,1000):
    try:
        # Produce line (without newline)
        line = lorem.sentence()
        p.produce("words", key=str(uuid4()), value=line)
        p.poll(0)
        p.flush()
        time.sleep(0.5)
        print(line)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

## Wikipedia edit stream

In [9]:
user_types = {True: 'bot', False: 'human'}
def init_namespaces():
    # create a dictionary for the various known namespaces
    # more info https://en.wikipedia.org/wiki/Wikipedia:Namespace#Programming
    namespace_dict = {-2: 'Media', 
                      -1: 'Special', 
                      0: 'main namespace', 
                      1: 'Talk', 
                      2: 'User', 3: 'User Talk',
                      4: 'Wikipedia', 5: 'Wikipedia Talk', 
                      6: 'File', 7: 'File Talk',
                      8: 'MediaWiki', 9: 'MediaWiki Talk', 
                      10: 'Template', 11: 'Template Talk', 
                      12: 'Help', 13: 'Help Talk', 
                      14: 'Category', 15: 'Category Talk', 
                      100: 'Portal', 101: 'Portal Talk',
                      108: 'Book', 109: 'Book Talk', 
                      118: 'Draft', 119: 'Draft Talk', 
                      446: 'Education Program', 447: 'Education Program Talk', 
                      710: 'TimedText', 711: 'TimedText Talk', 
                      828: 'Module', 829: 'Module Talk', 
                      2300: 'Gadget', 2301: 'Gadget Talk', 
                      2302: 'Gadget definition', 2303: 'Gadget definition Talk'}

    return namespace_dict


def construct_event(event_data, user_types):
    # use dictionary to change assign namespace value and catch any unknown namespaces (like ns 104)
    try:
        event_data['namespace'] = namespace_dict[event_data['namespace']]
    except KeyError:
        event_data['namespace'] = 'unknown'

    # assign user type value to either bot or human
    user_type = user_types[event_data['bot']]

    # define the structure of the json event that will be published to kafka topic
    event = {"id": event_data['id'],
             "domain": event_data['meta']['domain'],
             "uri": event_data['meta']['uri'],
             "namespace": event_data['namespace'],
             "title": event_data['title'],
             #"comment": event_data['comment'],
             "timestamp": event_data['meta']['dt'],#event_data['timestamp'],
             "user_name": event_data['user'],
             "user_type": user_type
             #"minor": event_datab['minor'],
             }
    
    if event_data["type"] == "edit":
        event["old_length"] = event_data['length']['old']
        event["new_length"] = event_data['length']['new']
    
    return event

In [ ]:
import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
namespace_dict = init_namespaces()
for event in EventSource(url):
    if event.event == 'message':
        try:
            event_data = json.loads(event.data)
        except ValueError:
            pass
        else:
            if event_data['type'] == 'edit':
                    # construct valid json event
                    event_to_send = construct_event(event_data, user_types)
                    #print(event_to_send)
                    p.produce('edit', value=json.dumps(event_to_send))
                    p.poll(0)
                    p.flush()

In [10]:
import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
namespace_dict = init_namespaces()
for event in EventSource(url):
    if event.event == 'message':
        try:
            event_data = json.loads(event.data)
        except ValueError:
            pass
        else:
            if event_data['type'] != 'log':
                # construct valid json event
                event_to_send = construct_event(event_data, user_types)
                #print(event_to_send)
                p.produce(event_data['type'], value=json.dumps(event_to_send))
                p.poll(0)
                p.flush()

KeyboardInterrupt: 